In [1]:
%%time
import malaya

CPU times: user 4.91 s, sys: 1.19 s, total: 6.1 s
Wall time: 7.16 s


## Wordvector augmentation

Let say you have a very limited labelled corpus, and you want to add more, but labelling is very costly.

So, text augmentation! You can use wordvector to replace words with similar semantics!

```python
def wordvector_augmentation(
    string,
    wordvector,
    threshold = 0.5,
    top_n = 5,
    soft = False,
    cleaning_function = None,
):
    """
    augmenting a string using wordvector.

    Parameters
    ----------
    string: str
    wordvector: object
        wordvector interface object.
    threshold: float, optional (default=0.5)
        random selection for a word.
    soft: bool, optional (default=False)
        if True, a word not in the dictionary will be replaced with nearest jarowrinkler ratio.
        if False, it will throw an exception if a word not in the dictionary.
    top_n: int, (default=5)
        number of nearest neighbors returned.
    cleaning_function: function, (default=None)
        function to clean text.

    Returns
    -------
    result: list
    """
```  

In [2]:
string = 'saya suka makan ayam dan ikan'

In [3]:
vocab_wiki, embedded_wiki = malaya.wordvector.load_wiki()
word_vector_wiki = malaya.wordvector.load(embedded_wiki, vocab_wiki)

In [4]:
augmented = malaya.generator.wordvector_augmentation(string,
                                  word_vector_wiki,
                                  soft=True)
augmented

['saya suka makan ayam dan ikan',
 'saya suka makan ayam serta ayam',
 'saya suka makan ayam atau ular',
 'saya suka makan ayam mahupun keju',
 'saya suka makan ayam tetapi lembu']

In [5]:
text = 'Perdana Menteri berkata, beliau perlu memperoleh maklumat terperinci berhubung isu berkenaan sebelum kerajaan dapat mengambil sebarang tindakan lanjut. Bagaimanapun, beliau yakin masalah itu dapat diselesaikan dan pentadbiran kerajaan boleh berfungsi dengan baik.'

In [6]:
augmented = malaya.generator.wordvector_augmentation(text,
                                  word_vector_wiki,
                                  soft=True)
augmented

['Perdana Menteri berkata , beliau perlu memperoleh maklumat terperinci berhubung isu berkenaan sebelum kerajaan dapat mengambil sebarang tindakan lanjut . Bagaimanapun , beliau yakin masalah itu dapat diselesaikan dan pentadbiran kerajaan boleh berfungsi dengan baik .',
 'Perdana Menteri berkata , beliau perlu memperoleh data terperinci berhubung isu berkaitan sebelum pemerintah dapat mendapat segala tindakan lanjut . Bagaimanapun , dia sedar masalah itu boleh diselesaikan serta pentadbiran pemerintah boleh berfungsi dengan baik .',
 'Perdana Menteri berkata , beliau perlu memperoleh bacaan terperinci berhubung isu tertentu sebelum perlembagaan dapat menghabiskan sesuatu tindakan lanjut . Bagaimanapun , baginda bimbang masalah itu harus diselesaikan atau pentadbiran perlembagaan boleh berfungsi dengan baik .',
 'Perdana Menteri berkata , beliau perlu memperoleh penjelasan terperinci berhubung isu tersebut sebelum kesultanan dapat mengubah suatu tindakan lanjut . Bagaimanapun , mereka 

## Transformer augmentation

Problem with wordvector, it just replaced a word for near synonym without understood the whole sentence context, so, Transformer comes to the rescue!

```python
def transformer_augmentation(
    string,
    model,
    threshold = 0.5,
    top_p = 0.8,
    top_k = 100,
    temperature = 0.8,
    top_n = 5,
    cleaning_function = None,
):

    """
    augmenting a string using transformer + nucleus sampling / top-k sampling.

    Parameters
    ----------
    string: str
    model: object
        transformer interface object. Right now only supported BERT.
    threshold: float, optional (default=0.5)
        random selection for a word.
    top_p: float, optional (default=0.8)
        cumulative sum of probabilities to sample a word. If top_n bigger than 0, the model will use nucleus sampling, else top-k sampling.
    top_k: int, optional (default=100)
        k for top-k sampling.
    temperature: float, optional (default=0.8)
        logits * temperature.
    top_n: int, (default=5)
        number of nearest neighbors returned.
    cleaning_function: function, (default=None)
        function to clean text.

    Returns
    -------
    result: list
    """
```    

In [7]:
model = malaya.transformer.load(model = 'albert')


INFO:tensorflow:loading sentence piece model





Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.





INFO:tensorflow:Restoring parameters from /Users/huseinzolkepli/Malaya/albert-model/base/albert-base/model.ckpt


In [8]:
augmented = malaya.generator.transformer_augmentation(text, model)
augmented

['Perdana Menteri berkata, pihaknya perlu memperoleh maklumat terperinci berhubung isu berkenaan sebelum kerajaan tidak menyelesaikan alih tindakan lanjut. Bagaimanapun, beliau yakin masalah itu dapat diselesaikan dan semua kerajaan boleh pulih dengan baik.',
 'Perdana Menteri berkata, kerajaan perlu memperoleh maklumat terperinci daripada masalah berkenaan sebelum kerajaan tidak mengendalikan alih tindakan lanjut. Bagaimanapun, beliau berharap masalah itu dapat diselesaikan kerana semua kerajaan boleh diselesaikan dengan baik.',
 'Perdana Menteri berkata, kerajaan perlu memperoleh maklumat terperinci daripada masalah berkenaan sebelum kerajaan tidak menyelesaikan alih tindakan lanjut. Bagaimanapun, beliau yakin masalah itu dapat diselesaikan supaya semua kerajaan boleh diselesaikan dengan baik.',
 'Perdana Menteri berkata, pihaknya perlu memperoleh maklumat terperinci mengenai isu berkenaan sebelum kerajaan tidak mengendalikan alih tindakan lanjut. Bagaimanapun, beliau berharap masala